In [482]:
import requests, bs4, re
from bs4 import BeautifulSoup as bs

In [483]:
url = 'https://www.autotrader.co.uk/car-details/202105202850513?model=911&onesearchad=New&onesearchad=Nearly%20New&onesearchad=Used&make=Porsche&radius=1500&year-to=2021&include-delivery-option=on&postcode=bh205lg&sort=price-desc&advertising-location=at_cars&page=1'

In [484]:
# url = 'https://www.autotrader.co.uk/car-details/202110048110259?year-to=2021&advertising-location=at_cars&model=911&make=Porsche&include-delivery-option=on&postcode=bh205lg&onesearchad=New&onesearchad=Nearly%20New&onesearchad=Used&radius=1501&sort=relevance&page=1'

In [485]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, os

chromedriver = "/Users/william.bell/opt/anaconda3/envs/metis/lib/python3.8/site-packages/chromedriver_binary/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [486]:
# query = "data science"
# youtube_search = "https://www.youtube.com/results?search_query="
# youtube_query = youtube_search + query.replace(' ', '+')
driver = webdriver.Chrome(chromedriver)
driver.implicitly_wait(10)
driver.get(url)
driver.maximize_window()

In [487]:
driver.page_source[:1000]

'<html lang="en" class="sp-message-open"><head><meta charset="utf-8">\n<meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n<meta name="theme-color" content="#313c53">\n\n<link rel="preconnect" href="https://ec.atcdn.co.uk" crossorigin="">\n<link rel="dns-prefetch" href="https://ec.atcdn.co.uk" crossorigin="" as="image">\n<link rel="preconnect" href="https://m.atcdn.co.uk" crossorigin="">\n<link rel="dns-prefetch" href="https://m.atcdn.co.uk" crossorigin="" as="image">\n\n<link rel="icon" href="https://www.autotrader.co.uk/images/favicons/favicon.png">\n<link rel="apple-touch-icon" href="https://www.autotrader.co.uk/images/favicons/touch-icon-iphone.png">\n<link rel="apple-touch-icon" sizes="76x76" href="https://www.autotrader.co.uk/images/favicons/touch-icon-ipad.png">\n<link rel="apple-touch-icon" sizes="120x120" href="https://www.autotrader.co.uk/images/favicons/touch-icon-iphone-retina.png">\n<link rel="apple-touch-icon" sizes="152x152" href="http

In [488]:
soup = bs(driver.page_source)

In [489]:
soup.find(id='content')

<main class="main__content used-fpa" id="content"><div class="skeleton-remove-fade"><div class="sc-bXDkGd dyMpem"><div class="sc-gsxoIA iYfbMn"><nav class="sc-fvhFOF jtqxYq"><a class="sc-fbNZge fGa-ddm atc-type-smart--medium" data-gui="back-to-search-link" href="/car-search?model=911&amp;onesearchad=New&amp;onesearchad=Nearly%20New&amp;onesearchad=Used&amp;make=Porsche&amp;radius=1500&amp;year-to=2021&amp;include-delivery-option=on&amp;postcode=bh205lg&amp;sort=price-desc&amp;advertising-location=at_cars&amp;page=1#202105202850513"><span class="icon-svg"><svg height="12" viewbox="0 0 13 12" width="13" xmlns="http://www.w3.org/2000/svg"><path d="M3.4 7H12a1 1 0 0 0 0-2H3.4l3.3-3.3a1 1 0 0 0 0-1.4 1 1 0 0 0-1.4 0l-5 5a1 1 0 0 0 0 1.4l5 5a1 1 0 0 0 1.4-1.4z"></path></svg></span><span class="sc-GTVdH ieBKgT">Back to results</span></a></nav></div></div><div class="sc-gsxoIA fjZDiT"><div class="sc-bsipwG jPEdib"><a class="sc-gInthZ cfhoHi atc-type-prius" data-gui="title-ctm-insurance-link" d

In [490]:
car = {}

In [491]:
miles_string = soup.find(id='ks-mileage').parent.find_next_sibling('span').text
amount_only = miles_string.split()[0]
amount_no_comma = amount_only.replace(',', '')
miles = int(amount_no_comma)
miles

6699

In [492]:
car['mileage'] = miles

In [493]:
price_string = soup.find(attrs={"data-testid": "advert-price"}).text
price_digits_only = price_string.replace(',', '').replace('£', '')
price = int(price_digits_only)
price

459950

In [494]:
car['price'] = price

In [495]:
advert_title = soup.find(attrs={"data-testid": "advert-title"})
name = advert_title.text
name

'Porsche 911'

In [496]:
car['name'] = name

In [ ]:
advert_subtitle = soup.find(attrs={"data-testid": "advert-subtitle"}).text
advert_subtitle

In [ ]:
car['advert_subtitle'] = advert_subtitle

In [497]:
reg_age = advert_title.find_next_sibling('p').text
year = reg_age.split()[0]
year

'2011'

In [498]:
car['year'] = year

In [499]:
key_specs = []
children = soup.find(attrs={"aria-label": "Key Specifications"}).findChildren("li")
for child in children:
   key_specs.append(child.text.strip())

In [500]:
car['body_type'] = key_specs[0]
car['engine_size'] = float(key_specs[1].replace("L", ""))
car['gearbox'] = key_specs[2]
car['fuel_type'] = key_specs[3]
car['doors'] = int(key_specs[4].split()[0])
car

{'mileage': 6699,
 'price': 459950,
 'name': 'Porsche 911',
 'year': '2011',
 'body_type': 'Coupe',
 'engine_size': 4.0,
 'gearbox': 'Manual',
 'fuel_type': 'Petrol',
 'doors': 2}

In [501]:
# driver.find_element_by_css_selector("*[title='Accept All']").text

In [502]:
# driver.find_element_by_xpath("//*[@id='notice']/div[3]/div[2]/button[2]").text

In [503]:
iframe = driver.find_element_by_xpath("//iframe[@title='SP Consent Message']")
driver.switch_to.frame(iframe)
driver.page_source[:10000]

'<html lang="en"><head>\n  <title>Notice Message App</title>\n  <meta charset="utf-8">\n  <meta name="viewport" content="width=device-width,initial-scale=1.0,maximum-scale=5.0,user-scalable=yes,viewport-fit=cover">\n  <meta name="mobile-web-app-capable" content="yes">\n  <meta name="apple-mobile-web-app-capable" content="yes">\n  <link rel="manifest" href="/manifest.json">\n  <style>body,html{font:14px/1.21 Helvetica Neue, arial, sans-serif;font-weight:400;margin:0;padding:0;min-height:100%;}</style>\n<link href="/Notice.03819.css" rel="preload" as="style"><style>body,html{font:14px/1.21 Helvetica Neue,arial,sans-serif;font-weight:400;margin:0;padding:0;height:100%;min-height:100%;text-size-adjust:none;-webkit-text-size-adjust:none;-moz-text-size-adjust:none;-ms-text-size-adjust:none;}body:not(.user-is-tabbing) :focus{outline:none;}</style><style>\n    .text-link-0-0:hover { color:#0000ff !important; text-decoration:underline !important; font-weight: 400 !important;  }\n  </style><styl

In [504]:
driver.find_element_by_css_selector("*[title='Accept All']").text

'Accept All'

In [505]:
driver.find_element_by_css_selector("*[title='Accept All']").click()

In [506]:
driver.switch_to.default_content()
driver.execute_script("window.scrollTo(0, 1080)")
driver.find_element_by_css_selector("*[data-testid='tech-spec-link']").text

'Specification'

In [507]:
driver.find_element_by_css_selector("*[data-testid='tech-spec-link']").click()

In [508]:
driver.page_source[:1000]

'<html lang="en" class="" tabindex="0"><head><meta charset="utf-8">\n<meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n<meta name="theme-color" content="#313c53">\n\n<link rel="preconnect" href="https://ec.atcdn.co.uk" crossorigin="">\n<link rel="dns-prefetch" href="https://ec.atcdn.co.uk" crossorigin="" as="image">\n<link rel="preconnect" href="https://m.atcdn.co.uk" crossorigin="">\n<link rel="dns-prefetch" href="https://m.atcdn.co.uk" crossorigin="" as="image">\n\n<link rel="icon" href="https://www.autotrader.co.uk/images/favicons/favicon.png">\n<link rel="apple-touch-icon" href="https://www.autotrader.co.uk/images/favicons/touch-icon-iphone.png">\n<link rel="apple-touch-icon" sizes="76x76" href="https://www.autotrader.co.uk/images/favicons/touch-icon-ipad.png">\n<link rel="apple-touch-icon" sizes="120x120" href="https://www.autotrader.co.uk/images/favicons/touch-icon-iphone-retina.png">\n<link rel="apple-touch-icon" sizes="152x152" href="https:

In [509]:
import time
time.sleep(3)

In [510]:
driver.page_source[:1000]

'<html lang="en" class=""><head><meta charset="utf-8">\n<meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n<meta name="theme-color" content="#313c53">\n\n<link rel="preconnect" href="https://ec.atcdn.co.uk" crossorigin="">\n<link rel="dns-prefetch" href="https://ec.atcdn.co.uk" crossorigin="" as="image">\n<link rel="preconnect" href="https://m.atcdn.co.uk" crossorigin="">\n<link rel="dns-prefetch" href="https://m.atcdn.co.uk" crossorigin="" as="image">\n\n<link rel="icon" href="https://www.autotrader.co.uk/images/favicons/favicon.png">\n<link rel="apple-touch-icon" href="https://www.autotrader.co.uk/images/favicons/touch-icon-iphone.png">\n<link rel="apple-touch-icon" sizes="76x76" href="https://www.autotrader.co.uk/images/favicons/touch-icon-ipad.png">\n<link rel="apple-touch-icon" sizes="120x120" href="https://www.autotrader.co.uk/images/favicons/touch-icon-iphone-retina.png">\n<link rel="apple-touch-icon" sizes="152x152" href="https://www.autotra

In [511]:
soup_spec_modal = bs(driver.page_source)

In [512]:
performance_expander = soup_spec_modal.find(attrs={"data-gui": "performance-expander"})
lines = performance_expander.findChildren()[2].find_all('li')

In [513]:
for line in lines:
   key = line.contents[0].text
   value = line.contents[1].text
   if "0-6" in key:
      car["0-6Xmph"] = float(value.split()[0])
   elif key == "Top Speed":
      car["top_speed"] = int(value.split()[0])
   elif key == "Cylinders":
      car["cylinders"] = int(value)
   elif key == "Engine power":
      car["power"] = int(value.split()[0])


car

{'mileage': 6699,
 'price': 459950,
 'name': 'Porsche 911',
 'year': '2011',
 'body_type': 'Coupe',
 'engine_size': 4.0,
 'gearbox': 'Manual',
 'fuel_type': 'Petrol',
 'doors': 2,
 '0-6Xmph': 3.9,
 'top_speed': 193,
 'cylinders': 6,
 'power': 500}

In [514]:
# record advert id
# subtitle?
# error messages etc



# 0-62mph
# 4.4 seconds
# Top Speed
# 181 mph
# Cylinders
# 6
# Valves
# 24
# Engine power
# 380 bhp
# Engine torque
# 332 lbs/ft
# driver.find_element_by_css_selector("*[title='Accept All']").text

In [515]:
# //*[@id="notice"]/div[3]/div[2]/button[2]

In [516]:
# title="SP Consent Message"

In [517]:
# driver.find_elements_by_name("iframe")

In [518]:
# driver.find_element_by_xpath("//iframe[@title='SP Consent Message']")